#About
In this project, I demonstrate the versatility and power of Large Language Models (LLMs) in automating and enhancing customer service operations for an auto dealership. By leveraging pre-trained models from Hugging Face, I've implemented a series of NLP tasks, including sentiment analysis of car reviews, customer question answering, text summarization, and translation. The dataset used comprises car reviews, highlighting customer opinions and experiences, which serves as the basis for our sentiment analysis and other NLP operations. This prototype not only showcases the potential for AI to support customer service and agent assistance but also illustrates how such technologies can be adapted across various sectors. * Data set is small for learning purposes.

# Data Dictionary

- Review: The full text of the customer review for a specific car model.
- Class: The sentiment class of the review (e.g., "POSITIVE" or "NEGATIVE").
- Make: The make of the car being reviewed (e.g., Nissan, Chevrolet).
- Model: The specific model of the car being reviewed (e.g., Rogue, NV).
- Year: The model year of the car being reviewed.


## Before you start

In order to complete the project you may wish to install some Hugging Face libraries such as `transformers` and `evaluate`.

In [6]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# Importing Libraries
Importing necessary libraries for data manipulation and machine learning.

In [7]:
!pip install transformers
!pip install evaluate

from transformers import logging
logging.set_verbosity(logging.WARNING)

import pandas as pd
import torch


#Loading the Dataset
Loading the car reviews dataset from a specified file path.

In [8]:
# Load the car reviews dataset
file_path = "/content/drive/MyDrive/Analyzing Car Reviews with LLMs/workspace/data/car_reviews.csv"
df = pd.read_csv(file_path, delimiter=";")


#Preparing Data
Extract reviews and their labels into lists for processing.

In [9]:
# Extract reviews and their sentiment labels into lists
reviews = df['Review'].tolist()
real_labels = df['Class'].tolist()


#Sentiment Classification with LLM
Perform sentiment analysis on the reviews using a pre-trained model

In [10]:
# Importing the pipeline function from transformers for sentiment analysis
from transformers import pipeline

# Loading a sentiment analysis model into the pipeline
classifier = pipeline('sentiment-analysis', model='distilbert-base-uncased-finetuned-sst-2-english')

# Predicting sentiment for each review and printing alongside actual sentiment
for review, prediction, label in zip(reviews, predicted_labels, real_labels):
    print(f"Review: {review}\nActual Sentiment: {label}\nPredicted Sentiment: {prediction['label']} (Confidence: {prediction['score']:.4f})\n")


Review: I am very satisfied with my 2014 Nissan NV SL. I use this van for my business deliveries and personal use. Camping, road trips, etc. We dont have any children so I store most of the seats in my warehouse. I wanted the passenger van for the rear air conditioning. We drove our van from Florida to California for a Cross Country trip in 2014. We averaged about 18 mpg. We drove thru a lot of rain and It was a very comfortable and stable vehicle. The V8 Nissan Titan engine is a 500k mile engine. It has been tested many times by delivery and trucking companies. This is why Nissan gives you a 5 year or 100k mile bumper to bumper warranty. Many people are scared about driving this van because of its size. But with front and rear sonar sensors, large mirrors and the back up camera. It is easy to drive. The front and rear sensors also monitor the front and rear sides of the bumpers making it easier to park close to objects. Our Nissan NV is a Tow Monster. It pulls our 5000 pound travel tr

#Evaluating Sentiment Classification
Calculate accuracy and F1 score for the sentiment predictions.

In [11]:
# Loading evaluation metrics
import evaluate

# Preparing data for accuracy and F1 calculation
references = [1 if label == "POSITIVE" else 0 for label in real_labels]
predictions = [1 if label['label'] == "POSITIVE" else 0 for label in predicted_labels]

# Calculating and printing accuracy and F1 score
accuracy_result = accuracy.compute(references=references, predictions=predictions)['accuracy']
f1_result = f1.compute(references=references, predictions=predictions)['f1']
print(f"Accuracy: {accuracy_result}\nF1 result: {f1_result}")


Accuracy: 0.8
F1 result: 0.8571428571428571


#Translation of Reviews
Translate the first review from English to Spanish using a translation model.

In [12]:
# Loading a translation model into the pipeline
translator = pipeline("translation", model="Helsinki-NLP/opus-mt-en-es")

# Translating the first review and displaying the result
translated_review = translator(reviews[0], max_length=27)[0]['translation_text']
print(f"Model translation:\n{translated_review}")


/usr/local/lib/python3.10/dist-packages/transformers/models/marian/tokenization_marian.py:197: UserWarning: Recommended: pip install sacremoses.
  warnings.warn("Recommended: pip install sacremoses.")
Your input_length: 365 is bigger than 0.9 * max_length: 27. You might consider increasing your max_length manually, e.g. translator('...', max_length=400)


Model translation:
Estoy muy satisfecho con mi 2014 Nissan NV SL. Uso esta furgoneta para mis entregas de negocios y uso personal.


#Evaluating Translation
Compare the model's translation with reference translations.

In [13]:
# Loading reference translations from a file
with open("/content/drive/MyDrive/Analyzing Car Reviews with LLMs/workspace/data/reference_translations.txt", 'r') as file:
    references = [line.strip() for line in file.readlines()]
print(f"Spanish translation references:\n{references}")

# Calculating and displaying BLEU score for translation quality
bleu_score = evaluate.load("bleu").compute(predictions=[translated_review], references=[references])['bleu']
print(f"BLEU score: {bleu_score}")


Spanish translation references:
['Estoy muy satisfecho con mi Nissan NV SL 2014. Utilizo esta camioneta para mis entregas comerciales y uso personal.', 'Estoy muy satisfecho con mi Nissan NV SL 2014. Uso esta furgoneta para mis entregas comerciales y uso personal.']
BLEU score: 0.6022774485691839


# Extractive Question Answering
Perform question answering on the context of a car review.

In [14]:
# Loading model and tokenizer for question answering
from transformers import AutoTokenizer, AutoModelForQuestionAnswering

model_ckp = "deepset/minilm-uncased-squad2"
tokenizer = AutoTokenizer.from_pretrained(model_ckp)
model = AutoModelForQuestionAnswering.from_pretrained(model_ckp)

# Preparing context, question, and performing tokenization
context = reviews[1]
question = "What did he like about the brand?"
inputs = tokenizer(question, context, return_tensors="pt")

# Performing inference to identify answer span
with torch.no_grad():
    outputs = model(**inputs)
start_idx, end_idx = torch.argmax(outputs.start_logits), torch.argmax(outputs.end_logits) + 1
answer_span = inputs["input_ids"][0][start_idx:end_idx]

# Decoding and displaying the answer
answer = tokenizer.decode(answer_span)
print(f"Answer: {answer}")


Some weights of the model checkpoint at deepset/minilm-uncased-squad2 were not used when initializing BertForQuestionAnswering: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
- This IS expected if you are initializing BertForQuestionAnswering from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForQuestionAnswering from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Answer: ride quality, reliability


#Text Summarization
Summarize the content of a car review.

In [15]:
# Loading a summarization model into the pipeline
summarizer = pipeline("summarization", model="cnicu/t5-small-booksum")

# Performing summarization on the last review
summarized_text = summarizer(reviews[-1], max_length=53)[0]['summary_text']
print(f"Summarized text:\n{summarized_text}")


Summarized text:
the Nissan Rogue provides me with the desired SUV experience without burdening me with an exorbitant payment; the financial arrangement is quite reasonable. I have hauled 12 bags of mulch in the back with the seats down and could have held more.
